In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [4]:
import pandas as pd
from tqdm import tqdm_notebook

In [5]:
df_all = pd.read_csv('../dataset/whichp450/train_other_data_added.csv',index_col=0)

In [6]:
df_all.head(2)
smi = df_all['Smiles'].tolist()

In [7]:
%%time
not_found = 0
can_smi = []
from rdkit import Chem
for smi in tqdm_notebook(smi):
    
    try:
        mol = Chem.MolFromSmiles(smi)
        cans = Chem.MolToSmiles(mol, True)
        can_smi.append(cans)
        
    except:
        can_smi.append('-')
        not_found = not_found + 1
        pass

# assert cans[0] == cans[1]


CPU times: user 1.94 s, sys: 80.7 ms, total: 2.02 s
Wall time: 2.18 s


In [8]:
not_found

0

In [9]:
df_all['Canonical_Smiles'] = can_smi

In [12]:
df = df_all
target_list=list(df['drug_type'].unique())
from tqdm import tqdm_notebook
# df = df_deduplicated
for k in tqdm_notebook(target_list):
    
    if k == '-':
        continue
        
    else:
    
        label = []

        for i in (range(len(df))):

            if df['drug_type'][i] == k:
                label.append('positive')
                continue

            else:
                label.append('negative')
                continue

        df['Label'] = label

        df_all = df

        df_all.reset_index(drop = True, inplace = True)

        drop_index = []

        for i in (range(len(df_all))):

            if df_all['drug_type'][i] == k:
                continue

            else:
                drop_index.append(i)
                continue

        df_p = df_all.drop(drop_index, axis = 0, inplace = False)
        df_p.reset_index(drop = True, inplace = True)

        df_pos = df_p.drop_duplicates(['Canonical_Smiles'])

        uni_pos_smiles = set(df_pos['Canonical_Smiles'].tolist())

        drop_index = []
        for i in (range(len(df_all))):

            if df_all['Canonical_Smiles'][i] in uni_pos_smiles:
                drop_index.append(i)
                continue

            else:
                continue

        df_neg = df_all.drop(drop_index, axis = 0, inplace = False)
        df_neg.reset_index(drop = True, inplace = True)

        df_neg = df_neg.drop_duplicates(['Canonical_Smiles'])

    #########################<GETTING FAM-SMILES-COUNT>####################################
        df_neg.reset_index(drop = True, inplace = True)
        df_pos.reset_index(drop = True, inplace = True)

    #     df_neg_index = df_neg.set_index('Final_Family')
    #     df_pos_index = df_pos.set_index('Final_Family')

    #     for fam in tqdm_notebook(fin_fam):
    #         get_fam_count(k, fam, df_neg_index, df_pos_index) 
    #######################################################################################

        clean_data = pd.concat([df_neg, df_pos], ignore_index=True, sort =True)

#         if k.split('-')[1] == 'Inhibitor':
#             clean_data.to_csv('../dataset/two_paper_plus_transformer_combined/inhib_data/' + k.split('-')[0] + '_inh.csv')
#             continue

#         elif k.split('-')[1] == 'Substrate':
#             clean_data.to_csv('../dataset/two_paper_plus_transformer_combined/sub_data/' + k.split('-')[0] + '_sub.csv')
            
#         else:
        print('No. of +ves for Enzyme '+k+' is: '+str(len(clean_data[clean_data.Label=='positive'])))
        clean_data.to_csv('../dataset/whichp450/individual_csv_files_plus_other/' + k+'.csv')
            


No. of +ves for Enzyme 1A2 is: 314
No. of +ves for Enzyme 2C19 is: 278
No. of +ves for Enzyme 2C8 is: 189
No. of +ves for Enzyme 2C9 is: 329
No. of +ves for Enzyme 2D6 is: 428
No. of +ves for Enzyme 2E1 is: 169
No. of +ves for Enzyme 3A4 is: 707

